# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [15]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice

import re
import pandas as pd

import itertools

'''
This function reads the html page used in exercises. It is used in multiple exercises.
Step to step what the function does.

1. Read html-page from saved local file. 

2. Then we filter page to contain all the RNA data.

3. Then we use regular expression to find all the data per RNA-triplet, aminoacid...

4. We store the data into pandas dataframe

5. Modify number column so we can turn that into integer

6. Convert column types to more appropriate
'''
def get_codong_usage_table_as_data_frame():
    #1. We read the html page
    try:
        with open('codon_usage_table.html') as f:
            page = f.read()
    except FileNotFoundError:
        with open('src/codon_usage_table.html') as f:
            page = f.read()

    #2. slice the information off the page (it is between <pre> and </pre> html-attributes)
    start_index = page.find('<pre>')+5
    end_index = page.find('</pre>')
    data = page[start_index:end_index].strip()

    #3. get an array containing information per RNA-triplet. Example after the next line data[0] = UUU F 0.46 17.6 (714298)
    data = re.findall(r'[a-zA-Z]{3}\s[a-zA-Z|*]{1}\s[0-9]*.[0-9]*\s*[0-9]*.[0-9]*\s*\(\s?[0-9]*\)', data)

    # we split the data into 2 dimensional array
    # Because some numbers has space inside the parentheses, we remove this space before splitting
    data = [d.replace('( ', '(').split() for d in data]

    #4. put everythin into pandas dataframe
    df = pd.DataFrame(data, columns=['triplet', 'amino acid',  'fraction', 'frequency', 'number'])

    #5 Remove parentheses in the number column
    df['number'] = df['number'].apply(lambda s: s[1:-1])

    #6 convert dtypes of fraction and frequency to float and number to int
    df = df.astype({'fraction':float, 'frequency':float, 'number':int})

    return df

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [16]:
dna_to_rna_dic = dict(A='A', C='C', G='G', T='U')

def dna_to_rna(s):
    return ''.join(dna_to_rna_dic[base] for base in s)
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution

Idea is that first we go troug a DNA-string to get each DNA-base. Then we get RNA-base for that DNA-base from dna_to_rna_dic dictionary. We join all of the RNA-bases and return the final RNA string.

### Discussion

Solution is quite simple. Most of the work was already done by the creator of this notebook. 

The use dict()-funtion to store symbol to symbol rules was given in the exercise so I included it in my solution.


## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [17]:

def get_dict():
    dic = dict()

    # get all the data
    df = get_codong_usage_table_as_data_frame()
    
    # create dictionay from iterable containing tuples of (triplet, amino acid)
    return dict(zip(df['triplet'],df['amino acid']))
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(len(codon_to_aa))


64


### Idea of solution

We read the data from the html page using the helper function. Then we create dictionay. We do this by creating (RNA-triplet, amino acid) tuples using pythons zip() function and making the series of tuples into dictionay. 

### Discussion
The program works fine. It returns dictionary that seems okay and contains all 64 RNA-amino acid combinations found in the html page. 

There is a helper function that reads the html page and returns a dataframe containing all the necessary data from the page. We will use this function also in later exercise. 

Step to step what the helper function does.

1. Read html-page from saved local file. 

2. Then we filter page to contain all the RNA data.

3. Then we use regular expression to find all the data per RNA-triplet, aminoacid...

4. We store the data into pandas dataframe

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [18]:

def get_dict_list():
    df = get_codong_usage_table_as_data_frame()

    dic = dict()

    # itereate tuples (amino acid, triplet)
    for t in zip(df['amino acid'], df['triplet']):
        amino_acid, triplet = t

        # if the amino acid is not in the dictionary, we add it
        if(amino_acid not in dic.keys()):
            dic[amino_acid] = [triplet]
        else:
            # else we add triplet to existing array
            dic[amino_acid] = dic[amino_acid]+[triplet]

    return dic
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    sum_of_values = 0
    for i in aa_to_codons.values():
        sum_of_values=sum_of_values+len(i)
    print(sum_of_values)

64


### Idea of solution

The idea is to save all amino acids as a key into dictionary, where values are an array of triplets. We can't use the exact same approach as in the exercise 2, because now there is different values for the same key. This is why we check is the key already in use and if it is then we add the RNA-triplet to existing array. This way we don't lose data with same keys.

### Discussion

The bacis idea is the same as in exercise 2. Note that we store only unique codons in a list. This is why the extra step to ensure that all the triplets are saved.
Solution takes care of the problem where we overwrite acid information with the latest triplet. As seen, there is 64 values in the dictionary.

Another way to solve this is to use function in exercise 1 and switch places of keys and values remembering that some aminoacids can be made with differend triplets.

Following pseudocode for alternate solution:

    def get_dict_list():
        dic = Dict()
        rna_to_aa = get_dict()
        for key, value in rna_to_aa.items():
            if value not in dic :
                dic[value] = key
            else
                dic[value] = list(set(dic[value] + key))
    return dic


With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [19]:

def rna_to_prot(s):
    rna_to_acid = get_dict()

    # Split RNA-sequence into groups of RNA-triplets
    rna = re.findall(r'...', s)

    #Create array containing amino acid for the RNA-triplet
    amino_acids = [rna_to_acid[r] for r in rna]

    # join amino acids into protein string
    return ''.join(amino_acids)

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

My solution is to change DNA-sequence into RNA-sequense. Then split RNA-sequense into RNA-triplets. After this my solution gets corresponding amino acids from the dictionary as a list. Finaly I join the amino acid-list to string and return it.

### Discussion

All test passed. Solution is the simplest I could think of. This is mostly because most of the hard work was already done in the previous exercises. All that was left to do was to split the RNA-sequence and change the aminoacids into wanted type.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function.

In [20]:

def get_probabability_dict():
    df = get_codong_usage_table_as_data_frame()
    amino_acids = df['amino acid']

    # Get the sum_aa for each amino acid
    sum_aa = dict()
    for aa in set(amino_acids):
        # use mask to get all the rows with same amino acid
        mask = df['amino acid'] == aa
        sum_aa[aa] = df.loc[mask, 'number'].sum()

    # Calculate the probability for each codon
    dic = dict()
    codons = df['triplet']
    df = df.set_index('triplet')
    for codon in codons:
        number = df.loc[codon, 'number']
        amino_acid = df.loc[codon, 'amino acid']
        dic[codon] =  number / sum_aa[amino_acid]
    return dic

if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print('\t'.join(f'{k}: {v:.6f}' for k, v in items[i*6:6+i*6]))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

The idea is to first calculate the sum from the number column for the same amino acid. Then we divide the number for each codon to its amino acid sum to get the overall ratio of that codon per all the same amino acid codons.


### Discussion

There are two ways to solve this. First and easy one is to read the fraction data from the table. In the second approach we are going to get the amount of how many times an amino acid appears. Then we calculate the fraction of that codon among the total amoun of the same amino acids.

We used the second approach since it is more accurate (and it passed the TMC test)


So we create a dictionay containing amino acid and how many times there are in total. 
Then for each codon we calculate the fraction of how many times it appears per thousand with the total amount of amino acid that codon creates.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [21]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    def convert(self, s):
        answer = []

        # dict of all the amino acids and their codons
        protein_rna = get_dict_list()
        # probability for each codon
        probability = get_probabability_dict()
    
        for protein in s:
            # all codons for amino acid
            codons = protein_rna[protein]

            # Then we get the codon with the highest probability
            max_codon = codons[0]
            max_value = probability[max_codon]
            for codon in codons:
                if(max_value < probability[codon]):
                    max_codon = codon
                    max_value = probability[codon]

            # add codon with highest probability to answer list
            answer.append(max_codon)

        # return string of all the codons
        return ''.join(answer)


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

Idea is that for each protein we find the codon with biggest fraction (probability) and add this codon to the list. Then we join the codon array into string.

### Discussion

With earlier exercises this one was quite straight forward. 

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [22]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    return choice(list(dist.keys()), p=list(dist.values()))

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

C, T, T, C, G, C, G, T, T, T, C, T, C, C, C, C, T, G, A, T, A, T, C, T, G, T, A, T, T


### Idea of solution

The idea is to use numpys random.choice. We give the keys and their probabilities and then return one of the keys by their probabilites.

### Discussion

All the test passed. Solution is quite simple with the help of telegram and google. At first I wasn't quite sure what I was supposed to do.

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [23]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass

    def convert(self, s):
        ans = []
        
        # get all possible codons for amino acid
        aa_codons = get_dict_list()

        # probability for each codon
        codons_probability = get_probabability_dict()

        for aa in s:
            codons = aa_codons[aa]

            # Then get probabilities for each codon
            prob_dict = dict()
            for codon in codons:
                prob_dict[codon] = codons_probability[codon]

            # Then picks one codon randomly using ex.7 function
            ans.append(random_event(prob_dict))

        # join all the codons into a string and return
        return ''.join(ans)
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

CUUACACCCAUCCAGAACCGCGCG


### Idea of solution

The idea is that we pick one codon by their probability for each amino acid in protein.

### Discussion

As stated in the description of the exercise the solution is quite simple.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [24]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    # start index for substring
    i = 0
    # unique characters in string s
    keys = set(s)

    for _ in s:
        # exit loop if over the end of the string
        if(len(s) < i+k):
            break

        # create dictionay with unique characters and their amount
        dic = dict.fromkeys(set(s), 0)

        # current substring if the string s
        substring = s[i:i+k]
        
        # count the amount of each character
        for char in set(substring):
            dic[char] = substring.count(char)

        # yield the dictionary
        yield dic

        # move to the next start point
        i = i+1
        
    
if __name__ == '__main__':
    s = "abc"
    for d in sliding_window(s, 3):
        print('-',d)

- {'a': 1, 'c': 1, 'b': 1}


### Idea of solution

Is to calculate the amount of characters found in the substring of the string. We iterate trough the string and break the loop after reaching the end of the string.

First we break the loop if the wanted substring goes 'over' the string. 

Then we create new dictionary with unique characters of the string.

Then we save the amoun of each character found in the substring and return the dictionary.

After yielding we increase the starting point of the substring.

### Discussion

After getting what the yield does it was simple to count the occurrence of characters in substring. 

The function does nothing if the k value is bigger than the length of the string. This is because in the TMC-test I got the idea that it is not necessary to start the count from beginning.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [25]:
def context_list(s, k):
    dic = dict()

    # get starting index for each possible substring
    for start_i in range(len(s)-k):

        # get substring length k+1
        substring = s[start_i:start_i+k+1]

        # make sure that substring is the rigth size
        if(len(substring)>k):
        # divide substring into key and value, 
        #   key is the k length substring
        #   value the character after this k length substring
            key, value = substring[0:-1], substring[-1]

            # create key in to dictionary if it dosn't exist
            if(key not in dic.keys()):
                dic[key] = ''

            # add value to result string
            dic[key] = dic[key] + value
    return dic

if __name__ == '__main__':
    k = 2
    s = 'ABC'
    d = context_list(s, 1)
    for key in d.keys():
        print(key, d[key])

A B
B C


### Idea of solution

The idea is to create all possible substrings with size of k+1. After this we slice the substring into to substrings. First sliced substring contains the key value (W in the description) and second substring value (c). Then we save this data into dictionary and return it.

### Discussion

My solution passed all the test. We could add something to handle the error in case k < 0. But I didn't do this. I think we can assume that k is always non negative

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [26]:
def context_probabilities(s, k):
    
    dic = dict()

    for key, value in context_list(s,k).items():
    
        prob_dict=dict()
        l = len(value)

        for char in set(value):
    
            # count the frequencie each character is found in the string
            amount = value.count(char)
    
            # save probability of that character 
            prob_dict[char] = amount / l
    
        # save probability of all characters to key value
        dic[key] = prob_dict
    
    return dic
    
if __name__ == '__main__':
    c = context_probabilities('ATGATATCATCGACGATGTAG',2)
    print(c)
    c = context_probabilities('ATGATATCATCGACGATGTAG',0)
    print(c)


{'AT': {'A': 0.2, 'G': 0.4, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'C': 0.3333333333333333, 'T': 0.6666666666666666}, 'TA': {'G': 0.5, 'T': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}
{'': {'A': 0.3333333333333333, 'G': 0.23809523809523808, 'C': 0.14285714285714285, 'T': 0.2857142857142857}}


### Idea of solution

Idea is to use the function from previous exercise.From here we calculate the frequency of each character in string and convert it to probability of that character appearing in the string. Then we save the probability dictionary into another dictionay. Now we have dictionay where context is the key to get the probability dictionary of each character.

### Discussion

All test passed. Solution it self is simple with previous exercise. All we need to do is to calculate each symbols frequency and turn that into probability of that symbol to appear in the context.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [27]:
class MarkovChain:
    def __init__(self, zeroth, kth, k=2):
        self.zeroth = zeroth
        self.kth = kth
        self.k = k

    def generate(self, n, seed=None):
        ans = ''
        rs = np.random.RandomState(seed=seed)
        keys = list(self.zeroth.keys())
        prob = list(self.zeroth.values())
        while(len(ans) < n):
            # Here we create the zeroth
            if(len(ans) < self.k):
                ans += rs.choice(keys, p=prob)
                continue
            # Last two DNA to get the next one
            last = ans[-2:]
            keys = list(self.kth[last].keys())
            prob = list(self.kth[last].values())

            ans += rs.choice(keys, p=prob)
        return ans

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {
    'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
    'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
    'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
    'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
    'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
    'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
    'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
    'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
    'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 5
    mc = MarkovChain(zeroth, kth)
    try:
        print(mc.generate(n, seed) == mc.generate(n, seed))
        print(mc.generate(n,seed))
    except KeyError:
        print('NOT SAME')

True
CGATCGATCA


### Idea of solution

Is to randomly pick k times random DNA from zeroth dictionay. After this we use kth dictionary using last k DNAs.


### Discussion

I was able to pick randomly using the seed value. I didn't use functions from prevoius exercise since I had no idea how to get same result every time we run the generator (with same n and seed value). This is because random event won't take seed value.

Also I didn't see any use for the above function(ex 11). 

Now my solution generates DNA sequence where first k DNA are selected randomly from the zeroth distribution and after this using the kmer distribution.

Not using the random_event made possible to get same results with same seed and length value.

Also note that there is a possibility, when picing the next DNA from the kmer distribution, that we run into key error. This is because in kmer distribution there isn't every possible option. Since we tackle this problem in the next exercise I didn't use any more time in this exercise. One possibility would have been to catch the keyError and pick one DNA randomly.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [68]:
def context_pseudo_probabilities(s, k):
    uniq = list(set(s))
    all_possible_keys = [''.join(tuple_of) for tuple_of in itertools.product(set(s), repeat=k)]

    # 11
    dic = dict()
    for key, value in context_list(s,k).items():
        prob_dict = dict()
        l = len(value) + 4
        for char in uniq:
            amount = value.count(char)
            prob_dict[char] = (amount + 1) / l
                
        dic[key] = prob_dict
    key_dict = dict(zip(uniq,np.full(len(uniq),1/len(uniq))))
    for key in all_possible_keys:
        if(key not in dic.keys()):
            dic[key] = key_dict
    return dic

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG" # AT -> G, A, C, C, G > 3 G, 2 a, 3 c, 1 t
    kth = context_pseudo_probabilities(s, 2)
    print(kth)
    #zeroth = context_pseudo_probabilities(s, 0)[""]
    #print(f"zeroth: {zeroth}")
    #print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    #print("\n", MarkovChain(zeroth, kth, k).generate(20))

{'AT': {'A': 0.2222222222222222, 'G': 0.3333333333333333, 'C': 0.3333333333333333, 'T': 0.1111111111111111}, 'TG': {'A': 0.3333333333333333, 'G': 0.16666666666666666, 'C': 0.16666666666666666, 'T': 0.3333333333333333}, 'GA': {'A': 0.14285714285714285, 'G': 0.14285714285714285, 'C': 0.2857142857142857, 'T': 0.42857142857142855}, 'TA': {'A': 0.16666666666666666, 'G': 0.3333333333333333, 'C': 0.16666666666666666, 'T': 0.3333333333333333}, 'TC': {'A': 0.3333333333333333, 'G': 0.3333333333333333, 'C': 0.16666666666666666, 'T': 0.16666666666666666}, 'CA': {'A': 0.2, 'G': 0.2, 'C': 0.2, 'T': 0.4}, 'CG': {'A': 0.5, 'G': 0.16666666666666666, 'C': 0.16666666666666666, 'T': 0.16666666666666666}, 'AC': {'A': 0.2, 'G': 0.4, 'C': 0.2, 'T': 0.2}, 'GT': {'A': 0.4, 'G': 0.2, 'C': 0.2, 'T': 0.2}, 'AA': {'A': 0.25, 'G': 0.25, 'C': 0.25, 'T': 0.25}, 'AG': {'A': 0.25, 'G': 0.25, 'C': 0.25, 'T': 0.25}, 'GG': {'A': 0.25, 'G': 0.25, 'C': 0.25, 'T': 0.25}, 'GC': {'A': 0.25, 'G': 0.25, 'C': 0.25, 'T': 0.25}, 'C

### Idea of solution

fill in 

### Discussion

fill in 

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [105]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        return np.nan

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATATCATCGACGATGTAG is nan


### Idea of solution

fill in

### Discussion

fill in

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [106]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        pass
        
    def log_probability(self, s):
        return np.nan
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is nan


### Idea of solution

fill in

### Discussion

fill in

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [107]:
def better_context_probabilities(s, k):
    return {"": ""}

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

: 


### Idea of solution

fill in

### Discussion

fill in

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [108]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        pass

    def generate(self, n, seed=None):
        return "Q"*n
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

QQQQQQQQQQ


### Idea of solution

fill in

### Discussion

fill in

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [109]:
def kmer_index(s, k):
    return {}

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{}


### Idea of solution

fill in

### Discussion

fill in

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [110]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    return {"".join(codon): 0 for codon in product("ACGU", repeat=3)}
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    return np.nan

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))    
    pass
    
    # Maybe check that converting back to protein results in the same sequence
    pass
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities("<rna sequence>") # placeholder call
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = {"".join(codon): 0 for codon in product("ACGU", repeat=3)} # placeholder dict
    
    # Create a completely random RNA sequence and get the codon probabilities
    pass
    cp_uniform = codon_probabilities("<random rna sequence>") # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

d(original || predicted) = nan
d(predicted || original) = nan

d(original || uniform) = nan
d(uniform || original) = nan

d(predicted || uniform) = nan
d(uniform || predicted) = nan


### Idea of solution

fill in

### Discussion

fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [111]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    return np.random.rand(2, 4) - 0.5
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

-0.044, -0.490, -0.452, -0.376
-0.047, -0.306, +0.105, +0.284


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [112]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[ 0.19469793 -0.48039796 -0.04533315  0.22531224]
 [ 0.1169615  -0.14184607 -0.4675314   0.05204944]]
Using [0.12, -0.14, -0.47, 0.05] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.99084387
KL divergence of stationary distribution prefix of length    10 is 0.55030422
KL divergence of stationary distribution prefix of length   100 is 0.23754124
KL divergence of stationary distribution prefix of length  1000 is 0.89511811
KL divergence of stationary distribution prefix of length 10000 is 0.98010139


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [113]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[ 0.20156918  0.24323634 -0.24088887 -0.44433302]

Using [ 0.19048416 -0.11956843  0.08329488  0.25969147] as initial distribution:
kl-divergence   empirical distribution
0.57058373813   [ 0.40963233 -0.14323095 -0.32774722 -0.12089462]
0.39012201235   [0.25164693 0.30803137 0.35829768 0.31388447]
0.40306038061   [-0.24886261 -0.42094969  0.07464634  0.05992421]
0.18344628210   [-0.39390102 -0.2086461  -0.27937    -0.17373437]
0.43507493010   [ 0.45885925 -0.15734219 -0.33781212  0.26357196]

Using [ 0.48146884 -0.22303988  0.0788808   0.19655225] as initial distribution:
kl-divergence   empirical distribution
0.17294369424   [-0.21781188 -0.29576814 -0.29731555  0.16453717]
0.24033765463   [ 0.45653142 -0.32562547 -0.3916086  -0.32628711]
0.14489875034   [-0.42946577 -0.37890945  0.28852276  0.13088767]
0.99985405962   [-0.33853882 -0.061

### Idea of solution

fill in

### Discussion
fill in